# Stochastic Gradient Descent

We now know how to use a neural network to make predictions, and how to train that network with regard to some input data.  However, to avoid numerical blowup, we need to learn pretty slowly for each iteration, and if we want to get anywhere, we'll need a lot more iterations.

To do that, we'll randomly subdivide our training data into lots of small pieces, and train on each piece.  We'll use all the data equally, but move a little based on each little piece.

Each training step is now very quick, but the extra overhead of subdividing and so on will mean each epoch now takes a fair amount longer.  It'll be worth it, I promise.

## Subdividing the Data

What we want to do is *randomly* divide the data into lots of little pieces called *mini-batches*.  We'll fix the size of the pieces in advance.  Typically, larger pieces mean fewer training steps per epoch (bad) but smaller pieces mean more randomness in each training step (also bad).  So we'll try to strike a balance, but it turns out not to be too big of a deal, so long as we randomize each epoch to avoid certain kinds of bias, and make our pieces large enough to avoid whole mini-batches of all one class (for example).

This is fairly easy to do with the `numpy.random.permutation` method:

In [1]:
def get_mini_batches(batch_size, X, Y):
    scrambled_indices = np.random.permutation(len(X))
    
    batch_edges = list(range(0, len(X), batch_size)) + [len(X)]
    num_batches = len(batch_edges)-1
    
    for i in range(0, num_batches):
        batch_indices = scrambled_indices[batch_edges[i]:batch_edges[i+1]]
        yield X[batch_indices], Y[batch_indices]

The above code will take in the batch size, as well as the training data (input and answers).  It scrambles the indices of the rows, and divides those indices into pieces of length `batch_size`, possibly with a last piece of smaller size if `X` doesn't divide evenly into mini-batches of the specified size.  Every row of `X` appears in exactly one mini-batch, and the order is random and different every time.

## Using Stochastic Gradient Descent

We can now adapt the code from the simulations before, where for each epoch, we cycle through all the mini-batches and train on them.  Let's use the MNIST data from before:

In [2]:
from mnist_import import get_mnist_nice

train_X, train_Y, test_X, test_Y = get_mnist_nice()

n = train_X.shape[1]
k = train_Y.shape[1]

We'll also use the same architecture as before, for the sake of a fair comparison:

In [3]:
from basic_nn import *

np.random.seed(31)

# Step 1: pick architecture (in prose above)
cost_function = cost_CE     # this is a classification problem
learning_rate = 0.125       # picked arbitrarily, seems to work okay

# Step 2: initialize

#100 input neurons, 100 neurons in a hidden layer, so 100+100+10 total
neuron_sizes = [100, 100]

weights, biases = initialize_xavier_sigmoid(n, k, neuron_sizes)
acts = [act_sigmoid for _ in range(0, len(weights))] # all sigmoid

In [4]:
# Step 3: train
import time

t1 = time.time()

num_epochs = 20
batch_size = 50

for epoch in range(0, num_epochs):
    # we'll keep track of the cost as we go
    total_cost = 0
    num_batches = 0
    
    for X_mb, Y_mb in get_mini_batches(batch_size, train_X, train_Y):
        x, z, y = forward_prop(weights, biases, acts, X_mb)

        bp_grad_w, bp_grad_b = back_prop(weights, biases, acts, cost_function, X_mb, Y_mb, x, y, z)

        for i in range(0, len(weights)):
            weights[i] -= learning_rate * bp_grad_w[i] / len(X_mb)
            biases[i] -= learning_rate * bp_grad_b[i] / len(X_mb)
    
        total_cost += cost_function(y[-1], Y_mb)
        num_batches += 1
    
    cost = total_cost / num_batches # average cost
    print("Cost {2:0.7f} through epoch {0}; took {1:0.3f} seconds so far.".format(epoch, time.time()-t1, cost))

Cost 2.8780716 through epoch 0; took 10.630 seconds so far.
Cost 1.8212673 through epoch 1; took 20.456 seconds so far.
Cost 1.2889390 through epoch 2; took 30.236 seconds so far.
Cost 1.0492573 through epoch 3; took 39.988 seconds so far.
Cost 0.9115346 through epoch 4; took 49.804 seconds so far.
Cost 0.8231766 through epoch 5; took 59.565 seconds so far.
Cost 0.7606813 through epoch 6; took 69.336 seconds so far.
Cost 0.7136595 through epoch 7; took 79.064 seconds so far.
Cost 0.6762746 through epoch 8; took 88.848 seconds so far.
Cost 0.6457295 through epoch 9; took 98.558 seconds so far.
Cost 0.6199872 through epoch 10; took 108.724 seconds so far.
Cost 0.5972880 through epoch 11; took 118.428 seconds so far.
Cost 0.5775128 through epoch 12; took 128.164 seconds so far.
Cost 0.5597899 through epoch 13; took 137.933 seconds so far.
Cost 0.5436318 through epoch 14; took 147.704 seconds so far.
Cost 0.5292783 through epoch 15; took 157.376 seconds so far.
Cost 0.5156977 through epoch

Some notes:
1. We now call it the error "through" epoch [n].  This is because we no longer get all the error at once, but get a little error, train a little, get a little more error, train a little more, and so on.
2. Epochs take a lot longer; they took about 5 seconds before, and now take about 10 seconds.
3. Each epoch accomplishes a lot lot more.

Let's do a comparison of classification error, as opposed to last time.

## Evaluating the Results

Let's look at classification error.  It's not as convenient as it was before to save the predictions at each epoch, since we didn't really construct them all at once.  But we can use the network we have now and evaluate its classification error.

In [5]:
_, _, y = forward_prop(weights, biases, acts, train_X)
success_rate = classification_success_rate(y[-1], train_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct.".format(num_epochs, 100*success_rate))

After 20 epochs, got 93.743% classifications correct.


That's not bad, right?  We'll do a lot better after we implement more bells and whistles (the state of the art is above 99%) but this is worlds ahead of our previous attempt, which got less than 12% correct.

## Why It Works

We are trying to do gradient descent, and fundamentally we are.  Unfortunately, computing the whole gradient is extremely time-consuming when the data is large.  What we do instead is take a *random sample* from the dataset, and compute the gradient using that small sample.  Since it's smaller, we can do it a lot more quickly.  However, since it doesn't include all the data, it could be slightly wrong; that is, this process introduces random noise.  This is why it's called *stochastic gradient descent*.

So in a nutshell, in the time it used to take to make two gradient steps, it can now do two thousand gradient steps, with a small amount of error at every stage.  Since we use all the data eventually, these errors balance each other out (sort of) and we end up going mostly in the right direction.